In [157]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.2'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [158]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import lpad, when, col

import time
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [159]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_df = home_df.withColumn("view",
    when(col("view") == 100, "100")
    .otherwise(lpad(col("view"), 2, '0'))
)
home_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|  00|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|  00|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|  04|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [160]:
# 2. Print our schema
home_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [161]:
# 3. Describe our data
home_df.describe().show()

+-------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|                  id|      date|        date_built|             price|          bedrooms|        bathrooms|       sqft_living|          sqft_lot|            floors|         waterfront|              view|
+-------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|  count|               33287|     33287|             33287|             33287|             33287|            33287|             33287|             33287|             33287|              33287|             33287|
|   mean|                NULL|      NULL|2013.5174091987863|314907.81797698804| 3.082224291765554|2.588758374140055|2116.5057529966653|11519.8677261

In [162]:
# 4. Temp View
home_df.createOrReplaceTempView("home_sales")

In [163]:
# 5. What is the average price for a four bedroom house sold in each year,
#    rounded to two decimal places?
avg_4bed_price = spark.sql("""SELECT year(date) as year,
  ROUND(AVG(price), 2) as avg_4bed_price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY year(date)
  ORDER BY year(date)""")

avg_4bed_price.show()

+----+--------------+
|year|avg_4bed_price|
+----+--------------+
|2019|      300263.7|
|2020|     298353.78|
|2021|     301819.44|
|2022|     296363.88|
+----+--------------+



In [164]:
# 6. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_3bd3ba2fl_20Ksf_price = spark.sql("""SELECT date_built as year, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY date_built
  ORDER BY date_built""")

avg_3bd3ba2fl_20Ksf_price.show()

+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [165]:
# 7. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Get the current time
start_time = time.time()

avg_rating_gte350K = spark.sql("""SELECT view AS view_rating,
  ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY view DESC""")

avg_rating_gte350K.show()

cashed_runtime = time.time() - start_time

# Print out the total runtime
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
|         80| 991767.38|
+-----------+----------+
only showing top 20 rows

--- 1.171095609664917 seconds ---


In [166]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [167]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [168]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_rating_gte350K = spark.sql("""SELECT view, 2, '0' AS view_rating,
  ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY view DESC""")

avg_rating_gte350K.show()

uncashed_runtime = time.time() - start_time

# Print out the total runtime
print(uncashed_runtime)

+----+---+-----------+----------+
|view|  2|view_rating| avg_price|
+----+---+-----------+----------+
|  99|  2|          0|1061201.42|
|  98|  2|          0|1053739.33|
|  97|  2|          0|1129040.15|
|  96|  2|          0|1017815.92|
|  95|  2|          0| 1054325.6|
|  94|  2|          0| 1033536.2|
|  93|  2|          0|1026006.06|
|  92|  2|          0| 970402.55|
|  91|  2|          0|1137372.73|
|  90|  2|          0|1062654.16|
|  89|  2|          0|1107839.15|
|  88|  2|          0|1031719.35|
|  87|  2|          0| 1072285.2|
|  86|  2|          0|1070444.25|
|  85|  2|          0|1056336.74|
|  84|  2|          0|1117233.13|
|  83|  2|          0|1033965.93|
|  82|  2|          0| 1063498.0|
|  81|  2|          0|1053472.79|
|  80|  2|          0| 991767.38|
+----+---+-----------+----------+
only showing top 20 rows

0.5136334896087646


In [169]:
#determine the runtime and compare to the original runtime
time_difference = cashed_runtime - uncashed_runtime
print(f"the uncached runtime of {uncashed_runtime:.4f} seconds was shorter than the cashed runtime of {cashed_runtime:.4f} by {time_difference:.4f} seconds")

the uncached runtime of 0.5136 seconds was shorter than the cashed runtime of 1.1709 by 0.6573 seconds


In [170]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")

In [171]:
# 11. Read the parquet formatted data.
parq_df = spark.read.parquet('home_partitioned')

In [172]:
# 12. Create a temporary table for the parquet data.
parq_df.createOrReplaceTempView('parq_sales')

In [173]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parq_df.createOrReplaceTempView('parq_sales')
avg_rating_gte350K2 = spark.sql("""SELECT view, 2, '0' AS view_rating,
  ROUND(AVG(price), 2) AS avg_price
  FROM parq_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY view DESC""")

avg_rating_gte350K2.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---+-----------+----------+
|view|  2|view_rating| avg_price|
+----+---+-----------+----------+
|  99|  2|          0|1061201.42|
|  98|  2|          0|1053739.33|
|  97|  2|          0|1129040.15|
|  96|  2|          0|1017815.92|
|  95|  2|          0| 1054325.6|
|  94|  2|          0| 1033536.2|
|  93|  2|          0|1026006.06|
|  92|  2|          0| 970402.55|
|  91|  2|          0|1137372.73|
|  90|  2|          0|1062654.16|
|  89|  2|          0|1107839.15|
|  88|  2|          0|1031719.35|
|  87|  2|          0| 1072285.2|
|  86|  2|          0|1070444.25|
|  85|  2|          0|1056336.74|
|  84|  2|          0|1117233.13|
|  83|  2|          0|1033965.93|
|  82|  2|          0| 1063498.0|
|  81|  2|          0|1053472.79|
|  80|  2|          0| 991767.38|
+----+---+-----------+----------+
only showing top 20 rows

--- 0.3687136173248291 seconds ---


In [174]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [175]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False